In [1]:
# using structure one from coenen et al. (2015)
# prior over causal graphs

import math
import numpy as np
from fractions import Fraction

In [2]:
n_hyp = 2
prior = [Fraction(1, n_hyp) for _ in range(n_hyp)]

Function to calculate prior entropy over graphs: $H(G) = \sum_{g \in G} P(g) \log_2 \frac{1}{P(g)}$

In [3]:
prior_entropy = Fraction(sum([p * math.log(1/p, 2) for p in prior]))
print(prior_entropy)

1


Function to calculate information gain based on a particular action and observed outcome: $$IG(a, o) = H(G) - H(G|a, o)$$

Function to calculate expected information gain for a particular action (averaging over all possible outcomes for that action): $$EIG(a) = H(G) - \sum_{o \in O} p(o|a)H(G|a, o)$$

Function to calculate posterior entropy: $$H(G|a, o) = \sum_{g \in G} p(g|a, o) \log_2 \frac{1}{P(g|a, o)}$$

Where $P(g|a, o)$ can be calculated using Bayes' rule as: $P(o|g, a)P(g)/P(o|a)$ and $P(o|a)$ by marginalizing over all graphs and their likelihood of producing outcome $o$ from action $a$

In [4]:
edges = np.array([[0, 0, 0], [1, 0, 1], [0, 0, 0]])
np.flatnonzero(edges[1])

array([0, 2])

In [6]:
from collections import deque

class DirectedGraph:
    def __init__(self, edges, transmission_rate=0.8, background_rate=0.05):
        self.adjacency_matrix = edges
        self.n = self.adjacency_matrix.shape[0]
        self.transmission_rate = transmission_rate
        self.background_rate = background_rate
        
        # TODO: add check to see if graph is not cyclic
        assert self.n >= 0
        assert self.transmission_rate >= 0.0
        
    def get_parents(self, node):
        """Calculate the parents of a given node"""
        return np.flatnonzero(self.adjacency_matrix[:, node])
        
    def get_children(self, node):
        """Calculate the children of a given node"""
        return np.flatnonzero(self.adjacency_matrix[node])
        
    def intervene(self, node):
        """Calculate the outcome from intervening on a particular node"""
        
        outcomes = np.zeros(self.n)  # array to store outcomes
        outcomes[node] = 1.0  # set intervened node to be on
        
        # temporarily remove edge between node and parent
        intervened_parents = self.get_parents(node) 
        self.adjacency_matrix[intervened_parents, node] = 0

        q = deque()  # create queue to store nodes
        
        # set root nodes to be bg rate and add to queue
        # root notes have no parents and are not the node intervened on
        for i in range(self.n):
            if len(self.get_parents(i)) == 0 and i != node:
                outcomes[i] = self.background_rate
                q.append(i)
        
        # append children of intervened node to queue
        children = self.get_children(node)
        q.append(children)  # append children of nodes to queue
        
        while len(q) is not 0:
            curr_node = q.popleft()  # remove first node from queue
            
            # calculate probability of turning on
            parents = self.get_parents(curr_node)
            outcomes[curr_node] = np.sum(outcomes[parents]) * \
                self.transmission_rate + self.background_rate
            
            # append children to queue
            children = self.get_children(curr_node)
            for child in children:
                q.append(child) 
        
        # add edges back in from parents to intervened node
        self.adjacency_matrix[intervened_parents, node] = 1

        # set any outcomes greater than 1 to 1
        outcomes[outcomes > 1.0] = 1.0
        
        return outcomes
        
    def likelihood(self):
        """Calculate the likelihood of a node being turned on?"""
        lik = np.zeros((self.n, self.n))
        for i in range(self.n):
            lik[i] = self.intervene(i)
            
        return lik

In [55]:
class ActiveGraphLearner:
    def __init__(self, graphs):
        self.n_hyp = len(graphs)
        self.actions = 3
        self.outcomes = 3
        self.hyp = graphs
        self.prior = 1 / self.n_hyp * np.ones((self.actions, self.outcomes))
        
    def likelihood(self):
        """Returns the likelihood of each action/outcome pair for each graph"""
        lik = np.array([h.likelihood() for h in self.hyp])
        return lik
    
    def update_posterior(self):
        """Calculates the posterior over all possible action/outcome pairs
        for each graph"""
        post = self.prior * self.likelihood()
        self.posterior = np.nan_to_num(post / np.sum(post, axis=0))
        
    def prior_entropy(self):
        pass
        
    def posterior_entropy(self):
        return np.nansum(self.posterior * np.log2(1 / self.posterior), axis=0)

In [56]:
edges_1 = np.array([[0, 0, 0], [1, 0, 1], [0, 0, 0]])  # common cause
edges_2 = np.array([[0, 1, 1], [0, 0, 0], [0, 0, 0]])  # common cause
dg1 = DirectedGraph(common_effect_1, transmission_rate=1.0, background_rate=0.05)
# dg2 = DirectedGraph(edges_2, transmission_rate=0.8, background_rate=0.05)
print(dg1.likelihood())
print(dg1.adjacency_matrix)
# graphs = [dg1, dg2]
# agl = ActiveGraphLearner(graphs)

appending 1
appending 2
appending 2
appending 1
[[ 1.    0.05  0.05]
 [ 1.    1.    0.05]
 [ 1.    0.05  1.  ]]
[[0 0 0]
 [1 0 0]
 [1 0 0]]


In [12]:
agl.update_posterior()
post = agl.posterior
eig = np.sum(post, axis=0) * agl.posterior_entropy()
print(eig)

[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]


//anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:18: RuntimeWarning: invalid value encountered in true_divide
//anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:24: RuntimeWarning: divide by zero encountered in true_divide
//anaconda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:24: RuntimeWarning: invalid value encountered in multiply


In [260]:
np.sum(post, axis=0)

array([[ 1.,  1.,  1.],
       [ 1.,  1.,  1.],
       [ 0.,  0.,  1.]])

In [15]:
# enumerate all graphs
common_cause_1 = np.array([[0, 1, 1], [0, 0, 0], [0, 0, 0]])
common_cause_2 = np.array([[0, 0, 0], [1, 0, 1], [0, 0, 0]])
common_cause_3 = np.array([[0, 0, 0], [0, 0, 0], [1, 1, 0]])
common_effect_1 = np.array([[0, 0, 1], [0, 0, 1], [0, 0, 0]])
common_effect_2 = np.array([[0, 1, 0], [0, 0, 0], [0, 1, 0]])
common_effect_3 = np.array([[0, 0, 0], [1, 0, 0], [1, 0, 0]])
causal_chain_1 = np.array([[0, 1, 0], [0, 0, 1], [0, 0, 0]])
causal_chain_2 = np.array([[0, 0, 0], [1, 0, 0], [0, 1, 0]])
causal_chain_3 = np.array([[0, 0, 1], [1, 0, 0], [0, 0, 0]])
causal_chain_4 = np.array([[0, 0, 0], [0, 0, 1], [1, 0, 0]])
causal_chain_5 = np.array([[0, 1, 0], [0, 0, 0], [1, 0, 0]])
causal_chain_6 = np.array([[0, 0, 1], [0, 0, 0], [0, 1, 0]])

In [25]:
foo = np.zeros(3, dtype='bool')
foo[2] = True
foo

array([False, False,  True], dtype=bool)